In [7]:
!pip install spacy pdfplumber pandas nltk



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 711.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 85.2 MB/s eta 0:00:00
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20240706
    Uninstalling pdfminer.six-20240706:
      Successfully uninstalled pdfminer.six-20240706


In [11]:
import pdfplumber
import spacy
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [13]:
# Load the spaCy model for NLP tasks
nlp = spacy.load("en_core_web_sm")

In [15]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text


In [17]:
def extract_name_and_contact(text):
    # Use regex to find the email and phone number
    name = ""
    email = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b', text)
    phone = re.findall(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', text)
    
    # Use spaCy for named entity recognition to find the name
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            name = ent.text
            break
    
    return name, email, phone


In [19]:
def extract_skills_and_experience(text):
    # Sample skills list (you can expand this based on your dataset)
    skills = ["Python", "Java", "JavaScript", "SQL", "C++", "Machine Learning", "Deep Learning", "Data Science"]
    skill_set = []
    for skill in skills:
        if skill.lower() in text.lower():
            skill_set.append(skill)
    
    # Extracting education and work experience
    education_keywords = ["Bachelor", "Master", "PhD", "University", "College"]
    work_experience_keywords = ["Experience", "Worked at", "Company", "Employment"]
    
    education = [sentence for sentence in text.split(".") if any(keyword in sentence for keyword in education_keywords)]
    work_experience = [sentence for sentence in text.split(".") if any(keyword in sentence for keyword in work_experience_keywords)]
    
    return skill_set, education, work_experience


In [21]:
def parse_resume(pdf_path):
    # Extract text from the resume
    text = extract_text_from_pdf(pdf_path)
    
    # Extract name, email, and phone
    name, email, phone = extract_name_and_contact(text)
    
    # Extract skills, education, and work experience
    skills, education, work_experience = extract_skills_and_experience(text)
    
    # Prepare a dictionary with extracted information
    resume_data = {
        "Name": name,
        "Email": email,
        "Phone": phone,
        "Skills": ", ".join(skills),
        "Education": " | ".join(education),
        "Work Experience": " | ".join(work_experience)
    }
    
    # Convert the data to a DataFrame
    resume_df = pd.DataFrame([resume_data])
    
    return resume_df


In [25]:
def extract_project_details(text):
    # Initialize the project details dictionary
    project_details = {
        "Project Description": "",
        "Team Size": "",
        "Operating System": "",
        "Languages / Technologies / Tools": "",
        "Role": "",
        "Responsibilities": ""
    }
    
    # Sample keywords for project sections
    project_keywords = ["Project", "Experience", "Responsibilities", "Technologies", "Role"]
    os_keywords = ["Windows", "Linux", "macOS", "Unix", "Android", "iOS", "Docker", "Kubernetes"]
    technologies_keywords = ["Python", "Java", "C#", "JavaScript", "SQL", "HTML", "CSS", "AWS", "Azure", "GCP", "React", "Node.js"]
    
    # Search for the project description
    if "project" in text.lower():
        project_details["Project Description"] = " | ".join([sentence for sentence in text.split(".") if "project" in sentence.lower()])
    
    # Search for the team size
    team_size = re.findall(r"\b\d{1,2}\s?person\b|\b\d{1,2}\s?members\b", text)
    project_details["Team Size"] = team_size[0] if team_size else "Not Mentioned"
    
    # Search for operating system mentions
    operating_systems = [os for os in os_keywords if os.lower() in text.lower()]
    project_details["Operating System"] = ", ".join(operating_systems) if operating_systems else "Not Mentioned"
    
    # Search for technologies/tools
    technologies = [tech for tech in technologies_keywords if tech.lower() in text.lower()]
    project_details["Languages / Technologies / Tools"] = ", ".join(technologies) if technologies else "Not Mentioned"
    
    # Search for the role
    role_keywords = ["Role", "Position", "Title"]
    roles = [sentence for sentence in text.split(".") if any(keyword.lower() in sentence.lower() for keyword in role_keywords)]
    project_details["Role"] = " | ".join(roles) if roles else "Not Mentioned"
    
    # Search for responsibilities
    responsibility_keywords = ["Responsibilities", "Task", "Duties"]
    responsibilities = [sentence for sentence in text.split(".") if any(keyword.lower() in sentence.lower() for keyword in responsibility_keywords)]
    project_details["Responsibilities"] = " | ".join(responsibilities) if responsibilities else "Not Mentioned"
    
    return project_details


In [31]:
def extract_multiple_projects(text):
    # Initialize a list to store all project details
    projects = []
    
    # Split the text into sections (assuming projects are separated by keywords like "Project")
    project_sections = [section.strip() for section in text.split("Project") if section.strip()]
    
    for section in project_sections:
        project_details = {
            "Project Description": "",
            "Team Size": "",
            "Operating System": "",
            "Languages / Technologies / Tools": "",
            "Role": "",
            "Responsibilities": ""
        }
        
        # Sample keywords for project sections
        os_keywords = ["Windows", "Linux", "macOS", "Unix", "Android", "iOS", "Docker", "Kubernetes"]
        technologies_keywords = ["Python", "Java", "C#", "JavaScript", "SQL", "HTML", "CSS", "AWS", "Azure", "GCP", "React", "Node.js"]
        
        # Extract project description (e.g., anything before 'Role' or 'Responsibilities')
        project_details["Project Description"] = " | ".join([sentence for sentence in section.split(".") if "project" in sentence.lower()])
        
        # Extract team size
        team_size = re.findall(r"\b\d{1,2}\s?person\b|\b\d{1,2}\s?members\b", section)
        project_details["Team Size"] = team_size[0] if team_size else "Not Mentioned"
        
        # Extract operating systems used
        operating_systems = [os for os in os_keywords if os.lower() in section.lower()]
        project_details["Operating System"] = ", ".join(operating_systems) if operating_systems else "Not Mentioned"
        
        # Extract technologies/tools used
        technologies = [tech for tech in technologies_keywords if tech.lower() in section.lower()]
        project_details["Languages / Technologies / Tools"] = ", ".join(technologies) if technologies else "Not Mentioned"
        
        # Extract role (e.g., look for words like 'Role', 'Position', 'Title')
        role_keywords = ["Role", "Position", "Title"]
        roles = [sentence for sentence in section.split(".") if any(keyword.lower() in sentence.lower() for keyword in role_keywords)]
        project_details["Role"] = " | ".join(roles) if roles else "Not Mentioned"
        
        # Extract responsibilities (e.g., look for words like 'Responsibilities', 'Tasks', 'Duties')
        responsibility_keywords = ["Responsibilities", "Task", "Duties"]
        responsibilities = [sentence for sentence in section.split(".") if any(keyword.lower() in sentence.lower() for keyword in responsibility_keywords)]
        project_details["Responsibilities"] = " | ".join(responsibilities) if responsibilities else "Not Mentioned"
        
        # Append the extracted project details to the projects list
        projects.append(project_details)
    
    return projects


In [33]:
def parse_resume(pdf_path):
    # Extract text from the resume
    text = extract_text_from_pdf(pdf_path)
    
    # Extract name, email, and phone
    name, email, phone = extract_name_and_contact(text)
    
    # Extract skills, education, and work experience
    skills, education, work_experience = extract_skills_and_experience(text)
    
    # Extract multiple projects
    projects = extract_multiple_projects(text)
    
    # Prepare the final resume data
    resume_data = {
        "Name": name,
        "Email": email,
        "Phone": phone,
        "Skills": ", ".join(skills),
        "Education": " | ".join(education),
        "Work Experience": " | ".join(work_experience),
        "Projects": projects  # List of project details
    }
    
    # Convert the data to a DataFrame for better visualization (optional)
    resume_df = pd.DataFrame([resume_data])
    
    return resume_df


In [35]:
# Specify the path to the resume PDF
pdf_path = "/Users/meghnadsaha/Downloads/resume/resume.pdf"

# Parse the resume and print the output
parsed_resume = parse_resume(pdf_path)
parsed_resume


,Name,Email,Phone,Skills,Education,Work Experience,Projects
0,• Worked,"[upendra.3m@gmail.com, upendra.3m@gmail.com, a...","[9966861242, 9966861242, 9966861242, 996686124...","Java, JavaScript, SQL",Tech from JNTU University\n10th (Sri Gowthami...,3 Years\n• Working at Cognizant technologies S...,"[{'Project Description': '', 'Team Size': 'Not..."


In [37]:
def display_projects(projects):
    # Create a DataFrame to display the extracted project details
    project_df = pd.DataFrame(projects)
    return project_df


In [41]:
# Define the function to extract text from PDF (if not already defined)
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Specify the path to the resume PDF
pdf_path = "/Users/meghnadsaha/Downloads/resume/resume.pdf"


# Extract text from the resume
text = extract_text_from_pdf(pdf_path)


In [43]:
# Now you can call the extract_multiple_projects function with the extracted text
projects = extract_multiple_projects(text)

# Optionally, you can display the extracted project details in a DataFrame
import pandas as pd

def display_projects(projects):
    # Create a DataFrame to display the extracted project details
    project_df = pd.DataFrame(projects)
    return project_df

# Display the projects
display_projects(projects)


,Project Description,Team Size,Operating System,Languages / Technologies / Tools,Role,Responsibilities
0,,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned
1,,Not Mentioned,Not Mentioned,"Java, JavaScript, SQL, HTML, CSS",Not Mentioned,Not Mentioned
2,"\n• Effectively co-coordinating, communicating...",Not Mentioned,"Windows, Linux","Java, JavaScript, HTML, CSS",Not Mentioned,Not Mentioned
3,,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned
4,,Not Mentioned,"Windows, Linux",Java,Not Mentioned,Not Mentioned
5,\nRole : Developer/Module Lead/Tech Lead\nResp...,Not Mentioned,Not Mentioned,Not Mentioned,\nRole : Developer/Module Lead/Tech Lead\nResp...,\nRole : Developer/Module Lead/Tech Lead\nResp...
6,,Not Mentioned,"Windows, Linux",Java,Not Mentioned,Not Mentioned
7,\nRole : Developer/Module Lead\nResponsibiliti...,Not Mentioned,Not Mentioned,Not Mentioned,\nRole : Developer/Module Lead\nResponsibiliti...,\nRole : Developer/Module Lead\nResponsibiliti...
8,,Not Mentioned,Windows,"Java, HTML",Not Mentioned,Not Mentioned
9,In this present project there were multiple j...,Not Mentioned,Not Mentioned,Java,\nRole : Developer/Module Lead\nResponsibiliti...,\nRole : Developer/Module Lead\nResponsibiliti...


In [45]:
import pdfplumber
import re
import pandas as pd

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Function to extract multiple projects
def extract_multiple_projects(text):
    # Initialize a list to store all project details
    projects = []
    
    # Split the text into sections (assuming projects are separated by keywords like "Project")
    project_sections = [section.strip() for section in text.split("Project") if section.strip()]
    
    for section in project_sections:
        project_details = {
            "Project Description": "",
            "Team Size": "",
            "Operating System": "",
            "Languages / Technologies / Tools": "",
            "Role": "",
            "Responsibilities": ""
        }
        
        # Sample keywords for project sections
        os_keywords = ["Windows", "Linux", "macOS", "Unix", "Android", "iOS", "Docker", "Kubernetes"]
        technologies_keywords = ["Python", "Java", "C#", "JavaScript", "SQL", "HTML", "CSS", "AWS", "Azure", "GCP", "React", "Node.js"]
        
        # Extract project description (e.g., anything before 'Role' or 'Responsibilities')
        project_details["Project Description"] = " | ".join([sentence for sentence in section.split(".") if "project" in sentence.lower()])
        
        # Extract team size
        team_size = re.findall(r"\b\d{1,2}\s?person\b|\b\d{1,2}\s?members\b", section)
        project_details["Team Size"] = team_size[0] if team_size else "Not Mentioned"
        
        # Extract operating systems used
        operating_systems = [os for os in os_keywords if os.lower() in section.lower()]
        project_details["Operating System"] = ", ".join(operating_systems) if operating_systems else "Not Mentioned"
        
        # Extract technologies/tools used
        technologies = [tech for tech in technologies_keywords if tech.lower() in section.lower()]
        project_details["Languages / Technologies / Tools"] = ", ".join(technologies) if technologies else "Not Mentioned"
        
        # Extract role (e.g., look for words like 'Role', 'Position', 'Title')
        role_keywords = ["Role", "Position", "Title"]
        roles = [sentence for sentence in section.split(".") if any(keyword.lower() in sentence.lower() for keyword in role_keywords)]
        project_details["Role"] = " | ".join(roles) if roles else "Not Mentioned"
        
        # Extract responsibilities (e.g., look for words like 'Responsibilities', 'Tasks', 'Duties')
        responsibility_keywords = ["Responsibilities", "Task", "Duties"]
        responsibilities = [sentence for sentence in section.split(".") if any(keyword.lower() in sentence.lower() for keyword in responsibility_keywords)]
        project_details["Responsibilities"] = " | ".join(responsibilities) if responsibilities else "Not Mentioned"
        
        # Append the extracted project details to the projects list
        projects.append(project_details)
    
    return projects

# Function to display projects in DataFrame format
def display_projects(projects):
    # Create a DataFrame to display the extracted project details
    project_df = pd.DataFrame(projects)
    return project_df

# Specify the path to the resume PDF
pdf_path = "/Users/meghnadsaha/Downloads/resume/resume.pdf"

# Extract text from the resume
text = extract_text_from_pdf(pdf_path)

# Extract multiple projects
projects = extract_multiple_projects(text)

# Display the extracted project details
display_projects(projects)


,Project Description,Team Size,Operating System,Languages / Technologies / Tools,Role,Responsibilities
0,,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned
1,,Not Mentioned,Not Mentioned,"Java, JavaScript, SQL, HTML, CSS",Not Mentioned,Not Mentioned
2,"\n• Effectively co-coordinating, communicating...",Not Mentioned,"Windows, Linux","Java, JavaScript, HTML, CSS",Not Mentioned,Not Mentioned
3,,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned,Not Mentioned
4,,Not Mentioned,"Windows, Linux",Java,Not Mentioned,Not Mentioned
5,\nRole : Developer/Module Lead/Tech Lead\nResp...,Not Mentioned,Not Mentioned,Not Mentioned,\nRole : Developer/Module Lead/Tech Lead\nResp...,\nRole : Developer/Module Lead/Tech Lead\nResp...
6,,Not Mentioned,"Windows, Linux",Java,Not Mentioned,Not Mentioned
7,\nRole : Developer/Module Lead\nResponsibiliti...,Not Mentioned,Not Mentioned,Not Mentioned,\nRole : Developer/Module Lead\nResponsibiliti...,\nRole : Developer/Module Lead\nResponsibiliti...
8,,Not Mentioned,Windows,"Java, HTML",Not Mentioned,Not Mentioned
9,In this present project there were multiple j...,Not Mentioned,Not Mentioned,Java,\nRole : Developer/Module Lead\nResponsibiliti...,\nRole : Developer/Module Lead\nResponsibiliti...


In [47]:
def export_projects_to_csv(projects, output_file="project_details.csv"):
    # Create a DataFrame from the project details
    project_df = pd.DataFrame(projects)
    
    # Export the DataFrame to a CSV file
    project_df.to_csv(output_file, index=False)
    print(f"Project details have been successfully exported to {output_file}")


In [89]:
# Specify the path to the resume PDF
pdf_path = "/IdeaProjects/Machine Learning/resume-perser/resume/Aditi Srivastava_RESUME1.pdf"


# Extract text from the resume
text = extract_text_from_pdf(pdf_path)

# Extract multiple projects
projects = extract_multiple_projects(text)

# Export the extracted project details to a CSV file
export_projects_to_csv(projects, "project_details.csv")


FileNotFoundError: [Errno 2] No such file or directory: '/IdeaProjects/Machine Learning/resume-perser/resume/Aditi Srivastava_RESUME1.pdf'

In [51]:
import json

def export_projects_to_json(projects, output_file="project_details.json"):
    # Convert the project details to a JSON string
    json_data = json.dumps(projects, indent=4)
    
    # Print the JSON data
    print(json_data)
    
    # Optionally, you can write it to a file
    with open(output_file, "w") as json_file:
        json_file.write(json_data)
        
    print(f"Project details have been successfully exported to {output_file}")


In [53]:
# Specify the path to the resume PDF
pdf_path = "/Users/meghnadsaha/Downloads/resume/resume.pdf"

# Extract text from the resume
text = extract_text_from_pdf(pdf_path)

# Extract multiple projects
projects = extract_multiple_projects(text)

# Print the extracted project details as JSON and export to a file
export_projects_to_json(projects, "project_details.json")


[
    {
        "Project Description": "",
        "Team Size": "Not Mentioned",
        "Operating System": "Not Mentioned",
        "Languages / Technologies / Tools": "Not Mentioned",
        "Role": "Not Mentioned",
        "Responsibilities": "Not Mentioned"
    },
    {
        "Project Description": "",
        "Team Size": "Not Mentioned",
        "Operating System": "Not Mentioned",
        "Languages / Technologies / Tools": "Java, JavaScript, SQL, HTML, CSS",
        "Role": "Not Mentioned",
        "Responsibilities": "Not Mentioned"
    },
    {
        "Project Description": "\n\u2022 Effectively co-coordinating, communicating and participating with the QA /Dev\nH1B Status Travelled United States as part of project deployment on clients location and had onsite\nworking experience",
        "Team Size": "Not Mentioned",
        "Operating System": "Windows, Linux",
        "Languages / Technologies / Tools": "Java, JavaScript, HTML, CSS",
        "Role": "Not Mentioned",
 

In [55]:
import re

def extract_multiple_projects(text):
    # Initialize a list to store all project details
    projects = []
    
    # Use a more specific pattern to match "Project" and extract sections more reliably
    # This assumes "Project" sections start with the word "Project" and end with a section separator like "Role", "Responsibilities", or "Skills"
    project_pattern = r"(Project[\s\S]*?)(?=(Role|Responsibilities|Skills|Education|Experience|$))"
    
    project_sections = re.findall(project_pattern, text)

    for section in project_sections:
        # The first element in each match is the project section
        project_details = {
            "Project Description": "",
            "Team Size": "",
            "Operating System": "",
            "Languages / Technologies / Tools": "",
            "Role": "",
            "Responsibilities": ""
        }
        
        # Extract the project description
        project_details["Project Description"] = section[0]
        
        # Extract team size
        team_size = re.findall(r"\b\d{1,2}\s?person\b|\b\d{1,2}\s?members\b", section[0])
        project_details["Team Size"] = team_size[0] if team_size else "Not Mentioned"
        
        # Extract operating systems used
        os_keywords = ["Windows", "Linux", "macOS", "Unix", "Android", "iOS", "Docker", "Kubernetes"]
        operating_systems = [os for os in os_keywords if os.lower() in section[0].lower()]
        project_details["Operating System"] = ", ".join(operating_systems) if operating_systems else "Not Mentioned"
        
        # Extract technologies/tools used
        technologies_keywords = ["Python", "Java", "C#", "JavaScript", "SQL", "HTML", "CSS", "AWS", "Azure", "GCP", "React", "Node.js"]
        technologies = [tech for tech in technologies_keywords if tech.lower() in section[0].lower()]
        project_details["Languages / Technologies / Tools"] = ", ".join(technologies) if technologies else "Not Mentioned"
        
        # Extract role (e.g., look for words like 'Role', 'Position', 'Title')
        role_keywords = ["Role", "Position", "Title"]
        roles = [sentence for sentence in section[0].split(".") if any(keyword.lower() in sentence.lower() for keyword in role_keywords)]
        project_details["Role"] = " | ".join(roles) if roles else "Not Mentioned"
        
        # Extract responsibilities (e.g., look for words like 'Responsibilities', 'Tasks', 'Duties')
        responsibility_keywords = ["Responsibilities", "Task", "Duties"]
        responsibilities = [sentence for sentence in section[0].split(".") if any(keyword.lower() in sentence.lower() for keyword in responsibility_keywords)]
        project_details["Responsibilities"] = " | ".join(responsibilities) if responsibilities else "Not Mentioned"
        
        # Append the extracted project details to the projects list
        projects.append(project_details)
    
    return projects


In [95]:
# Specify the path to the resume PDF
pdf_path = "/Users/meghnadsaha/Downloads/resume/resume.pdf"

# Extract text from the resume
text = extract_text_from_pdf(pdf_path)

# Extract multiple projects
projects = extract_multiple_projects(text)

# Print the extracted project details as JSON and export to a file
export_projects_to_json(projects, "project_details.json")


[
    {
        "Project Description": "upendra.3m@gmail.com\nMob: +91 9966861242\nName Jogarao Mantha\nEducational Qualification B. Tech from JNTU University\n10th (Sri Gowthami Public School) % - 90 %\n12th (Sri Chaitanya Junior College) % - 96 %\nGraduation (JNTU Kakinada) % \u2013 82.17\nH1B Visa\nCurrent Designation Sr Associate (Lead Technology)\u2013 Projects at Cognizant Technology Solutions\n(Cognizant)\n9.3 Years\n\u2022 Working at Cognizant technologies Solutions since Nov 2019 till date\nWork Experience \u2022 Worked at Wells Fargo Hyderabad since from 2018 to October 2019.\n | \u2022 Very good Knowledge of SDLC and Agile Process\n\u2022 Experience in Project Planning and Tracking activities\n\u2022 Handling Hot-fixes and Service packs based on the customer feedback/post-\nrelease QA activities.\n | \u2022 Review of Requirement, Analysis & Design, Use Case, User Interface\ndocuments.\n\u2022 Effectively co-coordinating, communicating and participating with the QA /Dev\nH1B 

In [86]:
# Specify the path to the resume PDF
pdf_path = "/Users/meghnadsaha/Downloads/resume/Aditi Srivastava_RESUME1.pdf"

# Extract text from the resume
text = extract_text_from_pdf(pdf_path)

# Extract multiple projects
projects = extract_multiple_projects(text)

# Print the extracted project details as JSON and export to a file
export_projects_to_json(projects, "project_details.json")


[
    {
        "Project Description": "Proficient in coordinating with the stakeholders to provide accurate reporting and information\nregarding the ongoing projects and initiatives. | Database Query : SQL Query\nWeb Server : Apache Tomcat, JBOSS,\nFrameworks : Struts, Hibernate, Spring Boot, Microservices,,Kafka,RAbbitMQ\nIDE\u2019s : My Eclipse & Dream Viewer\nUI : HTML,CSS,Bootstrap\nWork Profile\nLast worked in Buzzwork Business Services Pvt Ltd as a third party payroll from 23rd December\n2020 \u2013 March or Apr 2021\nWorked in the Project of SAP through Capgemini -SAP Successfactors.com-\nDescription SuccessFactors is a SAP product suite to provide cloud-based solution to manage various\nHR functions such as business alignment, people performance, recruitment, and learning activities for all\nsizes of organizations in more than 60 industries. | Project Type- Maintenance\nTechnologies : Java, J2EE ,Git,Commands\nFramework : Eclipse\nDatabase : | Did the maintenance activity and\